In [1]:
library(jsonlite)

final<- jsonlite::fromJSON("fall2019-survey-m08.json")

SyntaxError: invalid syntax (<ipython-input-1-3b967db7ee3f>, line 3)

In [2]:
#Cleaning data by removing the unnecessary columns irrelevant to the model
final <- final[,1:27] #eliminate last 5 column
final<- final[,-1:-2]#the following columns have nothing to do with airplane experiences.
final<- final[,-16:-18]
final<- final[,-13:-14]

SyntaxError: invalid syntax (<ipython-input-2-c643504a4623>, line 2)

In [3]:
#Converting likelihood.to recommend variable into binary to act as independent variable in logistic regression
value <- 1
threshold <- median(final$Likelihood.to.recommend)
value[final$Likelihood.to.recommend < threshold] <- 'NO'
value[final$Likelihood.to.recommend >= threshold] <- 'YES'
value <- as.factor(value)
final <- cbind(final,value)

SyntaxError: invalid syntax (<ipython-input-3-b1dce6de6021>, line 3)

In [4]:
#Munging data by modifying columns as per requirement and dealing with NA values

sortdep<- sort(final$Departure.Delay.in.Minutes,decreasing = TRUE)#sort departrue delay
meanDeDelay<- mean(head(sortdep,20))#claculate mean of top 20 departure delay
final$Departure.Delay.in.Minutes[which(is.na(final$Departure.Delay.in.Minutes))]<- meanDeDelay#fill NA with mean top 20 departure delay
sortArrDelay<- sort(final$Arrival.Delay.in.Minutes,decreasing = TRUE)#do again for arrival delay
meanArrDelay<- mean(head(sortArrDelay,20))#mean top 20 arrival delay
final$Arrival.Delay.in.Minutes[which(is.na(final$Arrival.Delay.in.Minutes))]<- meanArrDelay#fill NA with mean top 20 arrival delay

final$Year.of.First.Flight <- 2014 - final$Year.of.First.Flight
colnames(final)[5] <- 'Years.of.experience'

final$Scheduled.Departure.Hour[final$Scheduled.Departure.Hour == 5 | final$Scheduled.Departure.Hour == 6 | final$Scheduled.Departure.Hour == 7 | final$Scheduled.Departure.Hour == 8 | final$Scheduled.Departure.Hour == 9 | final$Scheduled.Departure.Hour == 10 | final$Scheduled.Departure.Hour == 11] <- 'Morning'
final$Scheduled.Departure.Hour[final$Scheduled.Departure.Hour == 12 | final$Scheduled.Departure.Hour == 13 | final$Scheduled.Departure.Hour == 14 | final$Scheduled.Departure.Hour == 15 | final$Scheduled.Departure.Hour == 16] <- 'Afternoon'
final$Scheduled.Departure.Hour[final$Scheduled.Departure.Hour == 17 | final$Scheduled.Departure.Hour == 18 | final$Scheduled.Departure.Hour == 19] <- 'Evening'
final$Scheduled.Departure.Hour[final$Scheduled.Departure.Hour == 20 | final$Scheduled.Departure.Hour == 21 | final$Scheduled.Departure.Hour == 22 | final$Scheduled.Departure.Hour == 23 | final$Scheduled.Departure.Hour == 14 | final$Scheduled.Departure.Hour == 1 | final$Scheduled.Departure.Hour == 2 | final$Scheduled.Departure.Hour == 3 | final$Scheduled.Departure.Hour == 4 ] <- 'Night'


SyntaxError: invalid syntax (<ipython-input-4-4b9726fd4e4b>, line 3)

In [5]:
#Changing data type to fit logistic regression

final$Airline.Status <- as.factor(final$Airline.Status)
final$Gender <- as.factor(final$Gender)
final$Price.Sensitivity <- as.factor(final$Price.Sensitivity)
final$Type.of.Travel <- as.factor(final$Type.of.Travel)
final$Class <- as.factor(final$Class)
final$Partner.Code <- as.factor(final$Partner.Code)
final$Scheduled.Departure.Hour <- as.factor(final$Scheduled.Departure.Hour)
final$Flight.cancelled <- as.factor(final$Flight.cancelled)

final$Age <- as.numeric(final$Age)
final$Flights.Per.Year <- as.numeric(final$Flights.Per.Year)
final$Total.Freq.Flyer.Accts <- as.numeric(final$Total.Freq.Flyer.Accts)
final$Shopping.Amount.at.Airport <- as.numeric(final$Shopping.Amount.at.Airport)
final$Eating.and.Drinking.at.Airport <- as.numeric(final$Eating.and.Drinking.at.Airport)
final$Departure.Delay.in.Minutes <- as.numeric(final$Departure.Delay.in.Minutes)
final$Arrival.Delay.in.Minutes <- as.numeric(final$Arrival.Delay.in.Minutes)
final$Flight.time.in.minutes <- as.numeric(final$Flight.time.in.minutes)
final$Flight.Distance <- as.numeric(final$Flight.Distance)
final$Likelihood.to.recommend <- as.numeric(final$Likelihood.to.recommend)

str(final)

SyntaxError: invalid syntax (<ipython-input-5-0a563271b1e6>, line 3)

In [6]:
#checking if Likelihood to recommend i.e. yes or no is represented by all levels of almost all categorical variables
xtabs(~ value + Airline.Status, data = final)
xtabs(~ value + Gender, data = final)
xtabs(~ value + Price.Sensitivity, data = final)
xtabs(~ value + Type.of.Travel, data = final)
xtabs(~ value + Total.Freq.Flyer.Accts, data = final)
xtabs(~ value + Class, data = final)
xtabs(~ value + Partner.Code, data = final)
xtabs(~ value + Scheduled.Departure.Hour, data = final)
xtabs(~ value + Flight.cancelled, data = final)

NameError: name 'xtabs' is not defined

In [7]:
GenderLogistic <- glm(value ~ Gender, data = final, family = "binomial")
summary(GenderLogistic)

TimeLogistic <- glm(value ~ Scheduled.Departure.Hour, data = final, family = "binomial")
summary(TimeLogistic)

SyntaxError: invalid syntax (<ipython-input-7-5296372333a0>, line 1)

In [8]:
Logistic1 <- glm(value ~ Loyalty+Airline.Status+Age+Gender + Flights.Per.Year + Total.Freq.Flyer.Accts + Type.of.Travel+Shopping.Amount.at.Airport+Eating.and.Drinking.at.Airport, data = final, family = "binomial")
summary(Logistic1)

library(caret)
trainIndex <- createDataPartition(final$value, p = .67,
                                  list = FALSE,
                                  times = 1)

Train <- final[ trainIndex,]
Test  <- final[-trainIndex,]

SyntaxError: invalid syntax (<ipython-input-8-d0db7c9023d3>, line 1)

In [9]:
library(tidyverse)

Test$model_prob <- predict(Logistic1, Test, type = "response")

Test <- Test  %>% mutate(model_pred = 1*(model_prob > .53) + 0,
                                 visit_binary = 1*(value == "YES") + 0)

Test <- Test %>% mutate(accurate = 1*(model_pred == visit_binary))
sum(Test$accurate)/nrow(Test)

SyntaxError: invalid syntax (<ipython-input-9-edeef5b38e71>, line 3)